In [1]:
from textblob import TextBlob 
from datetime import datetime
from pythainlp import word_tokenize
from nltk.corpus import stopwords
import nltk
import langdetect
import tweepy as tw
import pandas as pd
import re
import glob
import os
import schedule
import time
import requests

In [2]:
filename = glob.glob('*.csv')

In [3]:
nltk.download('stopwords')  
class DataManager:
    def __init__(self):
        ##-------------------- twitter --------------------##
        self._url = "https://api.aiforthai.in.th/ssense"                     
        self._headers = {'Apikey': "0kFkiFLdf4TAyY3JeUT9WVnB5naP6SjW"}
        consumer_key = "EaFU9nJw2utR0lo2PUmJE3VZy"
        consumer_secret = "DsZuVw0tEl6GHhyK08tunsOE9ICSfwplEhRDMQwB8VIqngZ6i8"
        access_token = "759317188863897600-nuwQmcYfDX8lvdRyw2eCD6fMRMkLzzZ"
        access_token_secret = 'zFFc5OJywNMBrRAblI7kFV62ZTZPHfTU1Q5kZ1cKzUupD'
        auth = tw.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        self._api = tw.API(auth, wait_on_rate_limit=True)

        self._url = "https://api.aiforthai.in.th/ssense"                     
        self._headers = {'Apikey': "0kFkiFLdf4TAyY3JeUT9WVnB5naP6SjW"}

        self.keys = []
        self.df = None
        self._start = 0

    def getSentimentENG(self,text):
        if TextBlob(text).sentiment.polarity > 0:
            return 'positive'
        elif TextBlob(text).sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

    def getSentimentTH(self,text):
        text = re.sub(r'[%]',' ',text)
        params = {'text':text}
        response = requests.get(self._url, headers=self._headers, params=params)
        try:
            polarity = str(response.json()['sentiment']['polarity'])
        except (KeyError):
            polarity = 'neutral'
        return polarity

    def formatdatetime(self,column):
        self.df[column] = pd.to_datetime(self.df[column]).dt.strftime('%Y/%m/%d') #dmY ทีหลัง
        self.df[column] = pd.to_datetime(self.df[column])
    
    def sortdf(self,columns):
        self.df.sort_values(by=columns,inplace=True)
        return self.df

    def unionfile(self,filenames):              #type filename -> list
        for file in filenames:
            df1 = pd.read_csv(file)
            if self._start != 0:
                self.df = pd.concat([self.df,df1])
                self.df.drop_duplicates(keep='last',inplace=True)
            else:
                self.df = df1
                self._start += 1
        return self.df
    
    def setnewDF(self,dataframe):
        self.df = dataframe
        return self.df

    def getperiod(self,since,until):  ####column for twitter
        self.formatdatetime('Time')
        dff = self.df
        dff.sort_values(by=['Time','Keyword'],inplace=True)
        if since == None and until != None:
            mask = (dff['Time']<=until)
        elif since != None and until == None:
            mask = (dff['Time']>=since)
        elif since != None and until != None:
            mask = (dff['Time']>=since) & (dff['Time']<=until)
        else:
            return
        return dff.loc[mask]

    def getrowwithkeys(self,keys):              #type keys -> list
        df = self.df
        return df.loc[df['Keyword'].isin(keys)]

    def collectword(self,dataframe):            #before edit collect thai can not use
        en_stops = set(stopwords.words('english'))
        word = {}
        for i in dataframe['Tweet']:    #only tweet
            if langdetect.detect(i) != 'th':
                allwords = i.split()
                for w in allwords: 
                    if w not in en_stops:
                        if w in word:
                            word[w] += 1
                        else:
                            word[w] = 1
            else:
                allwords = word_tokenize(i, engine='newmm')
                for w in allwords: 
                    if w not in en_stops:
                        if w in word:
                            word[w] += 1
                        else:
                            word[w] = 1
        del word['RT']
        del word[' ']   #for thai language
        sortword = sorted(word.items(),key=lambda x:x[1],reverse=True)
        return sortword     #tuple in list

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
dm = DataManager()
#dm.unionfile(filename)
dm.unionfile(['tweet_data_2732022.csv'])
dm.getperiod('2022-03-25',None)

,Keyword,User,Tweet,Language,Time,User Location,Hashtag,Polarity,Likes,Retweet,Sentiment
4050,shounen ai,myrialune,zoebug71 Also my favouriteeeee shounen ai ever...,en,2022-03-25,NaN,[],neutral,2,0,0.00
4051,shounen ai,MintFrogkie,abcdefghijklmvh cloudyyamilee SRY FOR LATE RES...,en,2022-03-25,NaN,[],negative,1,0,-0.30
4061,shounen ai,AthyChii,Boy Meets Maria Completed Shounen Ai,en,2022-03-25,NaN,[],neutral,12,1,0.00
4062,shounen ai,mkn0135,RT AthyChii Boy Meets Maria Completed Shounen Ai,en,2022-03-25,NaN,[],neutral,0,1,0.00
4063,shounen ai,ruinedlunarian,The only Gundam poster I own and it looks like...,en,2022-03-25,NaN,[],positive,0,0,0.30
...,...,...,...,...,...,...,...,...,...,...,...
4738,อนิเมะแนะนำ,ncwmqj,RT todojamy เธรดแนะนำรีวิว อนิเมะกีฬาที่เราเคย...,th,2022-03-26,NaN,['แนะนำอ'],positive,0,33,66.67
4876,อนิเมะแนะนำ,__RaInBoWLiGhT,RT btfeffectx ผมอยากแนะนำ EIGHTYSIX ให้ทุกคนดู...,th,2022-03-26,NaN,[],negative,0,9,66.67
4877,อนิเมะแนะนำ,lilinjeee,Teddix หนูขอแนะนำ Great pretender ค่ะ เป็นอนิเ...,th,2022-03-26,NaN,[],negative,0,0,75.00
4878,อนิเมะแนะนำ,ncwmqj,RT todojamy เธรดแนะนำรีวิว อนิเมะกีฬาที่เราเคย...,th,2022-03-26,NaN,['แนะนำอ'],positive,0,33,66.67


In [7]:
df1 = dm.getrowwithkeys(['ต่างโลก','pixar','shounen']).sort_values('Keyword')
for word in dm.collectword(df1)[:10]:
    print(word)

('นิยาย', 424)
('Pixar', 245)
('แฟนตาซี', 231)
('ตอน', 198)
('ใหม่', 191)
('ต่าง', 184)
('shounen', 179)
('โลก', 177)
('เด็กดี', 132)
('Algiz', 118)


In [ ]:
for word in dm.collectword(df1)[:10]:
    print(word)

In [ ]:
import pandas as pd
import re
import glob
filename = ['tweet_data_2032022.csv','tweet_data_1932022.csv','tweet_data_2132022_1.csv','tweet_data_2232022.csv']
i = 0
for file in filename:
    df1 = pd.read_csv(file)
    if i != 0:
        df = pd.concat([df,df1])
        df.drop_duplicates(keep='last',inplace=True)
    else:
        df = df1
        i += 1

df['Time'] = pd.to_datetime(df['Time']).dt.strftime('%Y/%m/%d')
#df['Time'] = pd.to_datetime(df['Time']).dt.strftime('%d %m %Y')
df['Time'] = pd.to_datetime(df['Time'])
df.sort_values(by=['Time','Keyword'],inplace=True)

#time = list(set(df['Time'].tolist()))
mask = (df['Time']>='2022-03-19') & (df['Time']<='2022-03-21')
#mask = (df['Time']>='19-03-2022') & (df['Time']<='21-03-2022')
df.loc[mask]

In [ ]:
en_stops = set(stopwords.words('english'))
filename = ['tweet_data_2732022.csv']
i = 0
for file in filename:
    df1 = pd.read_csv(file)
    if i != 0:
        df = pd.concat([df,df1])
        df.drop_duplicates(keep='last',inplace=True)
    else:
        df = df1
        i += 1
        
dff = df.loc[df['Keyword'].isin(['pixar','อนิเมะ'])]
word = {}

for i in dff['Tweet']:
    if langdetect.detect(i) != 'th':
        allwords = i.split()
        for w in allwords: 
            if w not in en_stops:
                if w in word:
                    word[w] += 1
                else:
                    word[w] = 1
    else:
        allwords = word_tokenize(i, engine='newmm')
        for w in allwords: 
            if w not in en_stops:
                if w in word:
                    word[w] += 1
                else:
                    word[w] = 1
del word['RT']
del word[' ']
sortword = sorted(word.items(),key=lambda x:x[1],reverse=True)
for i in sortword[:10]:
    print(i)

In [ ]:
from nltk.corpus import stopwords
en_stops = set(stopwords.words('english'))

all_words = ['There', 'is', 'a', 'tree','near','the','river']
for word in all_words: 
    if word not in en_stops:
        print(word)

In [ ]:
import requests
 
url = "https://api.aiforthai.in.th/ssense"
 
text = 'สาขานี้พนักงานน่ารักให้บริการดี'
 
data = {'text':text}
 
headers = {
    'Apikey': "0kFkiFLdf4TAyY3JeUT9WVnB5naP6SjW"
    }
 
response = requests.post(url, data=data, headers=headers)
 
print(response.json())
response.json()['preprocess']['keyword']

In [ ]:
key = list(set(df['Keyword'].tolist()))